In [36]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)
from scipy.stats import mode
import swifter

In [37]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv')

In [38]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)


In [39]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [40]:
agg = {
    "numerical_1":['sum','max','min','mean','var'],
    "numerical_2":['sum','max','min','mean','var'],
    "category_1":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "most_recent_sales_range":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "most_recent_purchases_range":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "avg_sales_lag3":['sum','max','min','mean','var'],
    "avg_purchases_lag3":['sum','max','min','mean','var'],
    "active_months_lag3":['sum','max','min','mean','var'],
    "avg_sales_lag6":['sum','max','min','mean','var'],
    "avg_purchases_lag6":['sum','max','min','mean','var'],
    "active_months_lag6":['sum','max','min','mean','var'],
    "avg_sales_lag12":['sum','max','min','mean','var'],    
    "avg_purchases_lag12":['sum','max','min','mean','var'],    
    "active_months_lag12":['sum','max','min','mean','var'],
    "category_4":{"mode" : lambda x : pd.Series.mode(x)},
    "category_2":{"mode" : lambda x : pd.Series.mode(x)},
    }

In [41]:
len(df_hist_trans.columns.tolist())

14

In [42]:
holidays = np.asarray([np.datetime64(date) for date in ['2017-01-01','2017-02-27','2017-02-28','2017-03-01','2017-04-14','2017-04-16','2017-04-21','2017-05-01','2017-06-15','2017-09-07','2017-10-12','2017-11-02','2017-11-15','2017-12-25','2018-01-01','2018-02-12','2018-02-13','2018-03-30','2018-04-21','2018-05-01','2018-05-31','2018-09-07','2018-10-12','2018-11-02','2018-11-15','2018-11-20','2018-12-25']])
seasons = [0,0,0,1,1,1,2,2,2,4,4,4]

def closest_holiday_days(x):
    deltas = holidays - np.datetime64(x)
    vect_abs = np.vectorize(abs)
    vect_days = np.vectorize(lambda x: x)
    absed = vect_abs(vect_days(deltas))
    return min(absed).item().days

In [43]:
def is_in_business_hours(x):
    if (x >= 9) and (x <= 20 ):
        return 1
    else:
        return 0   

def is_close_to_pay_day(x):
    if (x > 27) or (x < 4):
        return 1
    else:
        return 0

from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))    

In [44]:
is_close_to_pay_day(28)

1

In [45]:
%timeit closest_holiday_days(datetime.datetime.today())

281 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [46]:
import dask.dataframe as dd

# df_hist_trans = df_hist_trans.head(10000)
# df_new_merchant_trans = df_new_merchant_trans.head(10000)

tqdm.pandas()


for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])    
    print("processsed purchase_date")
    df['year'] = df['purchase_date'].dt.year
    print("year") 
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    print("wof")     
    df['month'] = df['purchase_date'].dt.month
    print("month")         
    df['season'] = df['month'].swifter.progress_bar(True).apply(lambda x:seasons[x-1])
    print("season")    
    df['closest_date_to_holiday'] = df['purchase_date'].swifter.progress_bar(True).apply(closest_holiday_days)
    print("holiday")             
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    print("dayofweek")    
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    print("weekend")        
    df['hour'] = df['purchase_date'].dt.hour
    print("hour")    
    df['weekofmonth'] = df['purchase_date'].swifter.progress_bar(True).apply(week_of_month)
    print('weekofmonth')    
#     df['is_in_business_hours'] = df['hour'].progress_apply(is_in_business_hours)
#     print('is_in_business_hours')
#     df['is_close_to_pay_day'] = df['month'].progress_apply(is_close_to_pay_day)
#     print('is_close_to_pay_day')    
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    print("AF")            
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    print("cat_1")    
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    print("month_diff")                
    df['month_diff'] += df['month_lag']

processsed purchase_date
year
wof
month



season



holiday
dayofweek
weekend
hour



weekofmonth
AF
cat_1
month_diff
processsed purchase_date
year
wof
month



season



holiday
dayofweek
weekend
hour



weekofmonth
AF
cat_1
month_diff


In [47]:
pd.set_option('display.width', -1)

In [64]:
df_hist_trans.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id',
 'year',
 'weekofyear',
 'month',
 'season',
 'closest_date_to_holiday',
 'dayofweek',
 'weekend',
 'hour',
 'weekofmonth',
 'month_diff',
 'category_2_mean',
 'category_3_mean']

In [49]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [50]:
def write_df_hist():
    df_hist_trans.to_csv("df_hist_trans.csv")

def new_merchant_trans():
    df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

In [51]:
from multiprocessing import Process
p = Process(target=write_df_hist)
p.start()

# df_hist_trans.to_csv("df_hist_trans.csv")
# df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

In [52]:
from multiprocessing import Process
p = Process(target=new_merchant_trans)
p.start()

In [53]:
# df_hist_trans = pd.read_csv("df_hist_trans.csv")

# df_new_merchant_trans = pd.read_csv("df_new_merchant_trans.csv")

In [54]:
# df_train = pd.read_csv('../input/train.csv')
# df_test = pd.read_csv('../input/test.csv')


In [61]:
pd.set_option('display.width', -1)

In [56]:
df_hist_trans['purchase_date'] = pd.to_datetime(df_hist_trans['purchase_date'])    
df_new_merchant_trans['purchase_date'] = pd.to_datetime(df_new_merchant_trans['purchase_date'])    

In [63]:
df_hist_trans.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id',
 'year',
 'weekofyear',
 'month',
 'season',
 'closest_date_to_holiday',
 'dayofweek',
 'weekend',
 'hour',
 'weekofmonth',
 'month_diff',
 'category_2_mean',
 'category_3_mean']

In [58]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','city_id','state_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
# aggs['is_in_business_hours'] = ['sum','max','min','mean','var']
# aggs['is_close_to_pay_day'] = ['sum','max','min','mean','var']
aggs["avg_sales_lag3"] = ['sum','max','min','mean','var']
aggs["avg_sales_lag6"] = ['sum','max','min','mean','var']
aggs['weekofmonth'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

KeyError: "Column 'avg_sales_lag3' does not exist!"

In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','city_id','state_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
# aggs['is_in_business_hours'] = ['sum','max','min','mean','var']
# aggs['is_close_to_pay_day'] = ['sum','max','min','mean','var']
aggs["avg_sales_lag3"] = ['sum','max','min','mean','var'],
aggs['weekofmonth'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

In [ ]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)


In [ ]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

In [ ]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)


In [59]:
def write_df_train():
    df_train.to_csv("elo_train.csv")

from multiprocessing import Process
p = Process(target=write_df_train)
p.start()

In [ ]:
def write_df_test():
    df_test.to_csv("elo_test.csv")

from multiprocessing import Process
p = Process(target=write_df_test)
p.start()

In [ ]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')


In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission_remove_few_features.csv", index=False)

In [11]:
pd.set_option('display.max_columns', None)

In [178]:
from scipy import stats

In [33]:
best_features.sort_values(by="importance",ascending=False).head(100)

,Feature,importance,fold
44,hist_month_diff_mean,2860,3
44,hist_month_diff_mean,2461,4
44,hist_month_diff_mean,2390,1
68,new_hist_purchase_amount_max,2363,3
46,hist_authorized_flag_mean,2327,3
68,new_hist_purchase_amount_max,2241,2
44,hist_month_diff_mean,2192,2
68,new_hist_purchase_amount_max,2100,1
68,new_hist_purchase_amount_max,2098,4
46,hist_authorized_flag_mean,2076,4


In [226]:
len(a)

60

In [171]:
mode([1,3,2,3,5])[0][0]

3

In [ ]:
# holidays = [1,3,2,3,5]
print(holidays)
a = [date for date in holidays]

In [ ]:
min(df_new_merchant_trans["purchase_date"])

In [ ]:
max(df_new_merchant_trans["purchase_date"])

In [ ]:
df_new_merchant_trans.dtypes

In [ ]:
[pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]]

In [ ]:
(pd.to_datetime("Apr 21, 2017") - pd.to_datetime('2017-01-01 00:00:08')).days

In [29]:
holidays = np.asarray([pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]])

In [34]:
[i.strftime("%Y-%m-%d") for i in holidays]

['2017-01-01',
 '2017-02-27',
 '2017-02-28',
 '2017-03-01',
 '2017-04-14',
 '2017-04-16',
 '2017-04-21',
 '2017-05-01',
 '2017-06-15',
 '2017-09-07',
 '2017-10-12',
 '2017-11-02',
 '2017-11-15',
 '2017-12-25',
 '2018-01-01',
 '2018-02-12',
 '2018-02-13',
 '2018-03-30',
 '2018-04-21',
 '2018-05-01',
 '2018-05-31',
 '2018-09-07',
 '2018-10-12',
 '2018-11-02',
 '2018-11-15',
 '2018-11-20',
 '2018-12-25']